In [2]:
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-macosx_11_0_arm64.whl size=122256 sha256=0340c2911a444999869707caff912384bdc6910ab81769e2be2d252224939234
  Stored in directory: /Users/yanicewan/Library/Caches/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [1]:
import os
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import pickle5 as pickle
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from keras.layers import Dense,Dropout,MaxPooling2D, Flatten, Conv2D

In [2]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    

In [24]:
with open("../preprocess_images/img_train.pkl", "rb") as fh:
    data = pickle.load(fh)
X_train_ = pd.DataFrame(data)["img_array"] 

with open("../preprocess_images/img_test.pkl", "rb") as fh:
    data = pickle.load(fh)
X_test_ = pd.DataFrame(data)["img_array"]

with open("../preprocess_images/img_y_train.pkl", "rb") as fh:
    data = pickle.load(fh)
y_train = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()

with open("../preprocess_images/img_y_test.pkl", "rb") as fh:
    data = pickle.load(fh)
y_test = np.array(pd.DataFrame(data)["label"].values.astype(np.float32)).flatten()

In [28]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# y_test[y_test == 2] = -1
# y_test[y_test == 1] = 2
# y_test[y_test == -1] = 1

# y_train[y_train == 2] = -1
# y_train[y_train == 1] = 2
# y_train[y_train == -1] = 1


X_train = []
X_test = []

for i in range(len(X_train_)):
    X_train.append(X_train_.values[i])
    
for i in range(len(X_test_)):
    X_test.append(X_test_.values[i])


X_train = np.array(X_train)
X_test = np.array(X_test)


acc = []
f1 = []
precision = []
recall = []
seeds = random.sample(range(1, 200), 1)
for seed in seeds:
    reset_random_seeds(seed)
    model = Sequential()
    model.add(Conv2D(100, (3, 3),  activation='relu', input_shape=(72, 72, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(100, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(3, activation = "softmax"))
    
    
    model.compile(Adam(learning_rate = 0.0001), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])
    
    model.summary()
    reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,        
    patience=8,      
    min_lr=1e-11,      
    verbose=1          
)

    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        batch_size=32,
        validation_split=0.1, 
        verbose=1,
        callbacks=[reduce_lr]  
    )

    # history = model.fit(X_train, y_train, epochs=50, batch_size=32,validation_split=0.1, verbose=1) 
    
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    acc.append(score[1])
    
    test_predictions = model.predict(X_test)
    test_label = to_categorical(y_test,3)

    true_label= np.argmax(test_label, axis =1)

    predicted_label= np.argmax(test_predictions, axis =1)
    
    cr = classification_report(true_label, predicted_label, output_dict=True)
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 70, 70, 100)       2800      
                                                                 
 max_pooling2d_40 (MaxPooli  (None, 35, 35, 100)       0         
 ng2D)                                                           
                                                                 
 dropout_44 (Dropout)        (None, 35, 35, 100)       0         
                                                                 
 conv2d_41 (Conv2D)          (None, 33, 33, 100)       90100     
                                                                 
 max_pooling2d_41 (MaxPooli  (None, 16, 16, 100)       0         
 ng2D)                                                           
                                                                 
 dropout_45 (Dropout)        (None, 16, 16, 100)     

In [31]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# y_test[y_test == 2] = -1
# y_test[y_test == 1] = 2
# y_test[y_test == -1] = 1

# y_train[y_train == 2] = -1
# y_train[y_train == 1] = 2
# y_train[y_train == -1] = 1


X_train = []
X_test = []

for i in range(len(X_train_)):
    X_train.append(X_train_.values[i])
    
for i in range(len(X_test_)):
    X_test.append(X_test_.values[i])


X_train = np.array(X_train)
X_test = np.array(X_test)


acc = []
f1 = []
precision = []
recall = []
seeds = random.sample(range(1, 200), 1)
for seed in seeds:
    reset_random_seeds(seed)
    model = Sequential()
    model.add(Conv2D(100, (3, 3),  activation='relu', input_shape=(72, 72, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(100, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(3, activation = "softmax"))
    
    
    model.compile(Adam(learning_rate = 0.0001), "sparse_categorical_crossentropy", metrics = ["sparse_categorical_accuracy"])
    
    model.summary()
    reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,        
    patience=8,      
    min_lr=1e-11,      
    verbose=1          
)

    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        batch_size=32,
        validation_split=0.1, 
        verbose=1,
        callbacks=[reduce_lr]  
    )

    # history = model.fit(X_train, y_train, epochs=50, batch_size=32,validation_split=0.1, verbose=1) 
    
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    acc.append(score[1])
    
    test_predictions = model.predict(X_test)
    test_label = to_categorical(y_test,3)

    true_label= np.argmax(test_label, axis =1)

    predicted_label= np.argmax(test_predictions, axis =1)
    
    cr = classification_report(true_label, predicted_label, output_dict=True)
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_47 (Conv2D)          (None, 70, 70, 100)       2800      
                                                                 
 max_pooling2d_47 (MaxPooli  (None, 35, 35, 100)       0         
 ng2D)                                                           
                                                                 
 dropout_52 (Dropout)        (None, 35, 35, 100)       0         
                                                                 
 conv2d_48 (Conv2D)          (None, 33, 33, 100)       90100     
                                                                 
 max_pooling2d_48 (MaxPooli  (None, 16, 16, 100)       0         
 ng2D)                                                           
                                                                 
 dropout_53 (Dropout)        (None, 16, 16, 100)     

2/2 [==============================] - 0s 18ms/step


In [9]:
print("Avg accuracy: " + str(np.array(acc).mean()))
print("Avg precision: " + str(np.array(precision).mean()))
print("Avg recall: " + str(np.array(recall).mean()))
print("Avg f1: " + str(np.array(f1).mean()))
print("Std accuracy: " + str(np.array(acc).std()))
print("Std precision: " + str(np.array(precision).std()))
print("Std recall: " + str(np.array(recall).std()))
print("Std f1: " + str(np.array(f1).std()))
print(acc)
print(precision)
print(recall)
print(f1)

Avg accuracy: 0.47956404089927673
Avg precision: 0.4103345594675074
Avg recall: 0.3411717681380602
Avg f1: 0.26167885841268396
Std accuracy: 0.0
Std precision: 0.0
Std recall: 0.0
Std f1: 0.0
[0.47956404089927673]
[0.4103345594675074]
[0.3411717681380602]
[0.26167885841268396]


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l2
import math

# 1. Setup data augmentation
datagen = ImageDataGenerator(
   rotation_range=15,
   width_shift_range=0.15,
   height_shift_range=0.15,
   horizontal_flip=True,
   vertical_flip=False,
   fill_mode='nearest',
   zoom_range=0.1,
   brightness_range=[0.9,1.1],
   validation_split=0.1
)

# 2. Create the enhanced model
model = Sequential([
   # First Block
   Conv2D(100, (3, 3), padding='same', kernel_regularizer=l2(0.01), input_shape=(72, 72, 3)),
   BatchNormalization(),
   Activation('relu'),
   MaxPooling2D((2, 2)),
   Dropout(0.3),
   
   # Second Block
   Conv2D(100, (3, 3), padding='same', kernel_regularizer=l2(0.01)),
   BatchNormalization(),
   Activation('relu'),
   MaxPooling2D((2, 2)),
   Dropout(0.3),
   
   # Third Block
   Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(0.01)),
   BatchNormalization(),
   Activation('relu'),
   MaxPooling2D((2, 2)),
   Dropout(0.3),
   
   # Dense Layers
   Flatten(),
   Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
   BatchNormalization(),
   Dropout(0.5),
   Dense(3, activation='softmax')
])

# 3. Learning rate scheduler with warmup and cosine decay
def cosine_decay_with_warmup(epoch, total_epochs, warmup_epochs, base_lr, min_lr):
   if epoch < warmup_epochs:
       return base_lr * (epoch + 1) / warmup_epochs
   
   progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
   return min_lr + 0.5 * (base_lr - min_lr) * (1 + math.cos(math.pi * progress))

lr_scheduler = LearningRateScheduler(
   lambda epoch: cosine_decay_with_warmup(
       epoch, 
       total_epochs=100, 
       warmup_epochs=5,
       base_lr=0.001,
       min_lr=1e-6
   )
)

# 4. Reduce learning rate plateau
reduce_lr = ReduceLROnPlateau(
   monitor='val_loss',
   factor=0.5,
   patience=8,
   min_lr=1e-6,
   verbose=1
)

# 5. Compile model
model.compile(
   optimizer=Adam(
       learning_rate=0.001,
       beta_1=0.9,
       beta_2=0.999,
       epsilon=1e-07,
       amsgrad=True
   ),
   loss='sparse_categorical_crossentropy',
   metrics=['sparse_categorical_accuracy']
)

# 6. Print model summary
model.summary()

# 7. Create data generators
train_generator = datagen.flow(
   X_train, y_train,
   batch_size=32,
   subset='training'
)

validation_generator = datagen.flow(
   X_train, y_train,
   batch_size=32,
   subset='validation'
)

# 8. Train the model
history = model.fit(
   train_generator,
   epochs=100,
   validation_data=validation_generator,
   callbacks=[lr_scheduler, reduce_lr],
   verbose=1
)

# 9. Evaluate model
score = model.evaluate(X_test, y_test, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# 10. Generate predictions and metrics
test_predictions = model.predict(X_test)
test_label = to_categorical(y_test, 3)
true_label = np.argmax(test_label, axis=1)
predicted_label = np.argmax(test_predictions, axis=1)

# 11. Calculate classification metrics
cr = classification_report(true_label, predicted_label, output_dict=True)
print("\nClassification Report:")
print(f"Precision: {cr['macro avg']['precision']:.4f}")
print(f"Recall: {cr['macro avg']['recall']:.4f}")
print(f"F1-score: {cr['macro avg']['f1-score']:.4f}")

# Store metrics if needed
precision.append(cr["macro avg"]["precision"])
recall.append(cr["macro avg"]["recall"])
f1.append(cr["macro avg"]["f1-score"])
acc.append(score[1])

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 72, 72, 100)       2800      
                                                                 
 batch_normalization_16 (Ba  (None, 72, 72, 100)       400       
 tchNormalization)                                               
                                                                 
 activation (Activation)     (None, 72, 72, 100)       0         
                                                                 
 max_pooling2d_44 (MaxPooli  (None, 36, 36, 100)       0         
 ng2D)                                                           
                                                                 
 dropout_48 (Dropout)        (None, 36, 36, 100)       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 36, 36, 100)     

2/2 [==============================] - 0s 21ms/step

Classification Report:
Precision: 0.5524
Recall: 0.5333
F1-score: 0.5144


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# 1. Set up data augmentation to increase training data diversity
datagen = ImageDataGenerator(
   rotation_range=10,
   width_shift_range=0.1,
   height_shift_range=0.1,
   horizontal_flip=True,
   fill_mode='nearest',
   validation_split=0.1  # Split ratio for validation set
)

# 2. Calculate class weights to handle class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# 3. Build enhanced model architecture
model = Sequential([
   # First Convolutional Block
   Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(72, 72, 3)),
   BatchNormalization(),  # Normalize activations for better training stability
   MaxPooling2D((2, 2)),
   Dropout(0.3),  # Prevent overfitting
   
   # Second Convolutional Block
   Conv2D(64, (3, 3), activation='relu', padding='same'),
   BatchNormalization(),
   MaxPooling2D((2, 2)),
   Dropout(0.3),
   
   # Third Convolutional Block
   Conv2D(32, (3, 3), activation='relu', padding='same'),
   BatchNormalization(),
   MaxPooling2D((2, 2)),
   Dropout(0.3),
   
   # Fully Connected Layers
   Flatten(),  # Convert 3D feature maps to 1D vector
   Dense(128, activation='relu'),  # Additional capacity for learning
   BatchNormalization(),
   Dropout(0.5),
   Dense(3, activation='softmax')  # Output layer for 3 classes
])

# 4. Setup learning rate scheduler for adaptive learning rate
reduce_lr = ReduceLROnPlateau(
   monitor='val_loss',  # Monitor validation loss
   factor=0.5,         # Multiply learning rate by 0.5 when plateauing
   patience=10,        # Number of epochs to wait before reducing LR
   min_lr=1e-10,       # Minimum learning rate
   verbose=1          # Print message when reducing LR
)

# 5. Compile model with optimizer and loss function
model.compile(
   optimizer=Adam(learning_rate=0.00005),
   loss='sparse_categorical_crossentropy',
   metrics=['sparse_categorical_accuracy']
)

# 6. Display model architecture
model.summary()

# 7. Create data generators for training and validation
train_generator = datagen.flow(
   X_train, y_train,
   batch_size=32,
   subset='training'
)

validation_generator = datagen.flow(
   X_train, y_train,
   batch_size=32,
   subset='validation'
)

# 8. Train model with augmented data and callbacks
history = model.fit(
   train_generator,
   epochs=100,
   validation_data=validation_generator,
   class_weight=class_weight_dict,  # Apply class weights
   callbacks=[reduce_lr]            # Use learning rate scheduler
)

# 9. Evaluate model performance on test set
score = model.evaluate(X_test, y_test, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# 10. Generate predictions and detailed metrics
test_predictions = model.predict(X_test)
predicted_label = np.argmax(test_predictions, axis=1)
test_label = to_categorical(y_test, 3)
true_label = np.argmax(test_label, axis=1)

# 11. Calculate and print classification metrics
cr = classification_report(true_label, predicted_label, output_dict=True)
print("\nClassification Report:")
print(f"Precision: {cr['macro avg']['precision']:.4f}")
print(f"Recall: {cr['macro avg']['recall']:.4f}")
print(f"F1-score: {cr['macro avg']['f1-score']:.4f}")

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 72, 72, 128)       3584      
                                                                 
 batch_normalization_12 (Ba  (None, 72, 72, 128)       512       
 tchNormalization)                                               
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 36, 36, 128)       0         
 ng2D)                                                           
                                                                 
 dropout_34 (Dropout)        (None, 36, 36, 128)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 36, 36, 64)        73792     
                                                                 
 batch_normalization_13 (Ba  (None, 36, 36, 64)      

KeyboardInterrupt: 